In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm 

from torch.utils.tensorboard import SummaryWriter

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset

from models.transformer_encoder import TransEncoder
from ddpm.ddpm import GaussianDiffusion1D
from evaluation.evaluation import KDEProgressionAnimator, vizual_comparison, plot_jsd_per_customer, plot_kde_samples

# Params

In [2]:
## Data
seq_len = 12
batch_size = 256

#NN
device = 'cuda' if torch.cuda.is_available() else 'cpu'
latent_dim = 256
cond_model = "mlp"
num_layers = 6
n_heads = 8
lr = 1e-4
betas = (0.9, 0.99)
epochs = 6
save_rate = 5

## DDPM
timesteps = 1000
beta_schedule = "cosine"
objective = "pred_noise"

## Logging
experiment_name = "trans_dev"
logging_dir = f"./logging/{experiment_name}/"

In [3]:
if not os.path.isdir(logging_dir):
    os.makedirs(logging_dir)
    os.makedirs(os.path.join(logging_dir, "viz/"))
    os.makedirs(os.path.join(logging_dir, "jsd/"))
    os.makedirs(os.path.join(logging_dir, "kde/"))
    os.makedirs(os.path.join(logging_dir, "tensorboard/"))
    os.makedirs(os.path.join(logging_dir, "weights/"))
    os.makedirs(os.path.join(logging_dir, "ts_sample/"))

# Utils

In [4]:
PREPROCESSED_DIR = "./preprocessing/data/customer_led_network_revolution/preprocessed/"

In [5]:
class MakeDATA(Dataset):
    def __init__(self, data, seq_len):
        data = np.asarray(data, dtype=np.float32)
        seq_data = []
        for i in range(len(data) - seq_len + 1):
            x = data[i : i + seq_len]
            seq_data.append(x)
        self.samples = np.asarray(seq_data, dtype=np.float32) 

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

# Load Data

In [ ]:
train = pd.read_csv(os.path.join(PREPROCESSED_DIR, "train.csv"))
val = pd.read_csv(os.path.join(PREPROCESSED_DIR, "val.csv"))
test = pd.read_csv(os.path.join(PREPROCESSED_DIR, "test.csv"))
cond_train = pd.read_csv(os.path.join(PREPROCESSED_DIR, "cond_train.csv"))
cond_val = pd.read_csv(os.path.join(PREPROCESSED_DIR, "cond_val.csv"))
cond_test = pd.read_csv(os.path.join(PREPROCESSED_DIR, "cond_test.csv"))

In [ ]:
train.drop("Date and Time of capture", axis=1, inplace=True)
val.drop("Date and Time of capture", axis=1, inplace=True)
test.drop("Date and Time of capture", axis=1, inplace=True)

In [ ]:
train.shape, val.shape, test.shape, cond_train.shape, cond_val.shape, cond_test.shape

In [ ]:
train_seq = np.asarray(MakeDATA(train, seq_len)).transpose(0, 2, 1)
cond_train_seq = np.asarray(MakeDATA(cond_train, seq_len)).transpose(0, 2, 1)

val_seq = np.asarray(MakeDATA(val, seq_len)).transpose(0, 2, 1)
cond_val_seq = np.asarray(MakeDATA(cond_val, seq_len)).transpose(0, 2, 1)

test_seq = np.asarray(MakeDATA(test, seq_len)).transpose(0, 2, 1)
cond_test_seq = np.asarray(MakeDATA(cond_test, seq_len)).transpose(0, 2, 1)

train_seq.shape, cond_train_seq.shape, test_seq.shape, cond_test_seq.shape

In [ ]:
train_dataset = TensorDataset(torch.from_numpy(train_seq), torch.from_numpy(cond_train_seq))
train_loader = DataLoader(train_dataset, batch_size, shuffle=False)

val_dataset = TensorDataset(torch.from_numpy(val_seq), torch.from_numpy(cond_val_seq))
val_loader = DataLoader(val_dataset, batch_size, shuffle=False)

test_dataset = TensorDataset(torch.from_numpy(test_seq), torch.from_numpy(cond_test_seq))
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

In [ ]:
real_data_val, real_cond_data_val = next(iter(val_loader))

# Load Model

In [ ]:
model = model = TransEncoder(features=train_seq.shape[1], latent_dim=latent_dim, num_heads=n_heads, num_layers=num_layers, cond_features=cond_train_seq.shape[1], cond_model=cond_model, device=device, seq_len=seq_len)

ddpm = GaussianDiffusion1D(model, seq_length=seq_len, timesteps=timesteps, objective=objective, loss_type='l2', beta_schedule=beta_schedule)
ddpm = ddpm.to(device)

optim = torch.optim.Adam(ddpm.parameters(), lr=lr, betas=betas)

writer = SummaryWriter(os.path.join(logging_dir, "tensorboard/"))

# Train

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    
    for i, (data, cond_data) in enumerate(train_loader):
        data = data.to(device)
        
        cond_data = cond_data.float()
        cond_data = cond_data.to(device)
        
        optim.zero_grad()
        
        loss = ddpm(data, cond_data)
        loss.backward()
        
        optim.step()
        running_loss += loss.item()
        
        if i % 100 == 0:
            last_loss = running_loss / 1000 # loss per batch
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
    return last_loss

In [ ]:
epoch_number = 0
best_val_loss = 1_000_000.
snapshots = {}
save_dir =  os.path.join(logging_dir, "weights/")

for epoch in tqdm(range(1, epochs+1)):
    print('epoch {}:'.format(epoch_number + 1))
    
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    
    running_val_loss = 0.0
    
    model.eval()
    with torch.no_grad():
        for i, (val_data, val_cond) in enumerate(val_loader):
            val_data = val_data.to(device)
        
            val_cond = val_cond.float()
            val_cond = val_cond.to(device)
            val_loss = ddpm(val_data, val_cond)
            
            running_val_loss += val_loss
    
    avg_val_loss = running_val_loss / (i + 1)
    print('Loss train {} val {}'.format(avg_loss, avg_val_loss))
    
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_val_loss },
                    epoch_number + 1)
    
    writer.flush()
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_epoch = epoch_number
        best_ddpm_model = ddpm.state_dict()
        best_optim_state_dict = optim.state_dict()
        
    
    if epoch_number % save_rate == 0:
        generated_sample = ddpm.sample(batch_size, real_cond_data_val.to(device))
        generated_sample = generated_sample.cpu().numpy()
        snapshots[epoch_number] = generated_sample
        
        model_name = f"model_epoch_{epoch_number}_val_{avg_val_loss:.4f}.pth"
        torch.save({
            'epoch': epoch_number,
            'diffusion_state_dict': ddpm.state_dict(),
            'diffusion_optim_state_dict': optim.state_dict()},
            os.path.join(save_dir, model_name))
    
    epoch_number += 1

In [ ]:
model_name = f"best_model.pth"
torch.save({
    'epoch': best_epoch,
    'diffusion_state_dict': best_ddpm_model,
    'diffusion_optim_state_dict': best_optim_state_dict},
    os.path.join(logging_dir, "weights/", model_name))

In [ ]:
for k, v in snapshots.items():
    plot_kde_samples(v, real_data_val,show=False, fpath=os.path.join(logging_dir, "kde/", f"kde_epoch_{k}.png"))

# Evaluation

In [ ]:
ddpm.load_state_dict(torch.load(os.path.join(logging_dir, "weights/", "best_model.pth"))["diffusion_state_dict"])

In [ ]:
real_data_test, real_cond_data_test = next(iter(test_loader))

In [ ]:
with torch.no_grad():
    samples = ddpm.sample(batch_size, real_cond_data_test.to(device))
    samples = samples.cpu().numpy()

print(f"Samples shape: {samples.shape}")

In [ ]:
vizual_comparison(samples, real_data_test, os.path.join(logging_dir, "viz/", "pca_umap_tsne.png"))

In [ ]:
plot_jsd_per_customer(samples, real_data_test, os.path.join(logging_dir, "jsd/", "jsd.png"))

In [ ]:
plot_kde_samples(samples, real_data_test, fpath=os.path.join(logging_dir, "kde/", f"kde.png"))

In [ ]:
batch_idx = np.random.randint(0, 256) 
customer_indices = np.random.randint(0, 256, size=15)
fig, axs = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

with torch.no_grad():
    sample = ddpm.sample(1, real_cond_data_test.to(device)) 
    sample = sample.cpu().numpy() 

for i in customer_indices:
    axs[0].plot(sample[0, i], alpha=0.7, label=f'Customer {i}')
    axs[1].plot(real_data_test[0, i], alpha=0.7, label=f'Customer {i}')

axs[0].set_title("Generated Data")
axs[1].set_title("Real Data")

for ax in axs:
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(logging_dir, "ts_sample/", "samples.png"))
plt.show()

In [ ]:
#Takes a long time but it works
animator = KDEProgressionAnimator(real_data_val.cpu().numpy(), snapshots.values())
animator.animate()
animator.save(os.path.join(logging_dir, "kde/", "kde_progression.gif"))